In [10]:
import re
import time
import os
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from string import Template
import urllib
import json
import pickle


from pprint import pprint
import sys

import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,davies_bouldin_score,calinski_harabasz_score
from sklearn.decomposition import PCA, SparsePCA
from sklearn.manifold import Isomap
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [11]:
pd.set_option('display.max_rows', 500)


## 對照表整併

In [16]:
## 經濟地理資訊平台API抓的資料

with open('/content/drive/MyDrive/03_RESOURCE/GIS/gis_open_api.pickle', 'rb') as f:
    api_data = pickle.load(f)

In [17]:

# 分行爬蟲資訊

## 各分行地址與基本資訊
brn_df = pd.read_csv('/content/drive/MyDrive/03_RESOURCE/GIS/taishin_brn.csv').drop_duplicates()

## 分行地址與點位
brn_xy_df = pd.read_csv('/content/drive/MyDrive/03_RESOURCE/GIS/brn_xy_df.csv')

## 合併並去掉重複
brn_df = brn_df.merge(brn_xy_df,how='left' ,on = 'addr').copy()
brn_df = brn_df.drop_duplicates()

## 作為主表
brn_code_df = brn_df[['brn', 'addr', 'lon', 'lat']]

## 整理經濟三級發布區
brn_code3_df = pd.DataFrame(
    [(x,api_data[x]['code3_data']['ADMIV']['CODE3']) for x in list(api_data.keys()) if x !='南崁分行'],
    columns = ['brn','eco_code3']
)
brn_code3_df = pd.concat([brn_code3_df, pd.DataFrame([['南崁分行','A6800005028']], columns=['brn','eco_code3'])])
brn_code3_df = brn_code3_df.reset_index(drop=True)

## 整理統計二級發布區
brn_code2_df = pd.read_csv('/content/drive/MyDrive/03_RESOURCE/GIS/分行統計區代碼對照.csv', encoding='CP950')
brn_code2_df = brn_code2_df[['ID','縣市', '鄉鎮市區', '村里', '二級發布區', '一級發布區', '最小統計區']].rename(
    {
      'ID': 'brn',
      '縣市':'city',
      '鄉鎮市區':'town',
      '村里':'village',
      '二級發布區':'code_2',
      '一級發布區':'code_1',
      '最小統計區':'code_min'
    },
    axis=1
)

# 整併
brn_code_df = brn_code_df.merge(brn_code2_df, how='left', on=['brn'])
brn_code_df = brn_code_df.merge(brn_code3_df, how='left', on=['brn'])

# 資料清理
## 處理異體字跟鄉鎮市區層級更名

brn_code_df['town'] = brn_code_df['town'].replace('員林鎮', '員林市')
brn_code_df.loc[brn_code_df['town']=='新店區','village'] = brn_code_df.loc[brn_code_df['town']=='新店區','village'].replace('五?里', '五峰里')


## 電信信令人口資料

In [22]:
## 電信信令人口資料
tele_df = pd.read_csv('/content/drive/MyDrive/03_RESOURCE/GIS/109年11月行政區電信信令人口統計資料_鄉鎮市區.csv', encoding='CP950',skiprows=0)

In [23]:
tele_chi_nm = tele_df.iloc[0,:].copy()
tele_chi_nm

COUNTY_ID                       縣市代碼
COUNTY                          縣市名稱
TOWN_ID                       鄉鎮市區代碼
TOWN                          鄉鎮市區名稱
NIGHT_WORK                  平日夜間停留人數
DAY_WORK(7:00~13:00)        平日上午活動人數
DAY_WORK(13:00~19:00)       平日下午活動人數
DAY_WORK                    平日日間活動人數
NIGHT_WEEKEND               假日夜間停留人數
DAY_WEEKEND(7:00~13:00)     假日上午活動人數
DAY_WEEKEND(13:00~19:00)    假日下午活動人數
DAY_WEEKEND                 假日日間活動人數
MORNING_WORK                  平日早晨旅次
MIDDAY_WORK                   平日中午旅次
AFTERNOON_WORK                平日午後旅次
EVENING_WORK                  平日晚上旅次
MORNING_WEEKEND               假日早晨旅次
MIDDAY_WEEKEND                假日中午旅次
AFTERNOON_WEEKEND             假日午後旅次
EVENING_WEEKEND               假日晚上旅次
INFO_TIME                       資料時間
Name: 0, dtype: object

In [24]:
tele_df = tele_df.iloc[1:,:].copy()

In [27]:
tele_df['day_night_ratio'] = tele_df['DAY_WORK(13:00~19:00)']/(tele_df['NIGHT_WORK']+1)

In [28]:
tele_df['holiday_work_ratio'] = tele_df['NIGHT_WORK']/(tele_df['DAY_WORK']+1)

In [30]:
tele_df = tele_df[['COUNTY','TOWN','day_night_ratio','holiday_work_ratio']]

In [31]:
tele_df[['day_night_ratio','holiday_work_ratio']] = tele_df[['day_night_ratio','holiday_work_ratio']].astype(float)

## 周圍公司行號

In [32]:
# 公司行號工廠資訊

poi_num_df = []
for brn in list(api_data.keys()):
    df = pd.DataFrame([list(api_data[brn]['around_num'].values())],columns=['factory_num','bu_num', 'cmp_num', 'stk_num'])
    poi_num_df.append(df)
poi_num_df = pd.concat(poi_num_df)
poi_num_df['brn'] = list(api_data.keys())

In [33]:
%%capture
'''
## openstreetmap資料
import pickle
with open('/content/drive/MyDrive/03_RESOURCE/GIS/brn_poi.pickle', 'rb') as f:
  poi = pickle.load(f)
'''

## 人口

In [34]:
## 人口資料

pop_ratio = pd.read_csv('/content/drive/MyDrive/03_RESOURCE/GIS/台灣二級統計區人口指標.csv')
pop_data = pd.read_csv('/content/drive/MyDrive/03_RESOURCE/GIS/台灣二級統計區人口資料.csv')

pop_ratio = pop_ratio.rename({'CODE2':'code_2'},axis=1)
pop_data = pop_data.rename({'CODE2':'code_2'},axis=1)


## 年收

In [35]:
## 年收入資料 
## 資料年度107年 單位千元
sal_df = pd.read_csv('https://www.fia.gov.tw/WEB/fia/ias/ias106/106_165-9.csv')

## 電子發票消費熱度

In [36]:
## 政府製作的消費熱度

e_inv_ratio = pd.read_csv('https://sip.einvoice.nat.gov.tw/ods-main/ODS308E/download/691C0280-CEFB-488F-9E71-6AA4F39A41CD/1/1124193D-09F5-4711-AB9A-01848E3B88E4/0/?fileType=csv')

In [37]:
## 刪除所有字串內空白

def whitespace_remover(dataframe):
    # iterating over the columns
    for i in dataframe.columns:        
        # checking datatype of each columns
        if dataframe[i].dtype == 'object':       
            # applying strip function on column
            dataframe[i] = dataframe[i].str.replace('\s+', '', regex=True)
        else:              
            # if condn. is False then it will do nothing.
            pass

In [38]:
e_inv_ratio = e_inv_ratio[e_inv_ratio['年度']==2020][['縣市','鄉鎮市區', '村里','主行業別', '消費熱度計算來源', '張數指標', '銷售額指標']]
e_inv_ratio.eval('綜合指標 = (張數指標+銷售額指標)/2', inplace=True)

## groupby mean計算不同計算來源

e_inv_ratio = e_inv_ratio.groupby(['縣市', '鄉鎮市區', '村里', '主行業別'])[['綜合指標']]\
.agg('mean')\
.reset_index(level=[0,1,2,3])

e_inv_ratio = e_inv_ratio.pivot_table(index=['縣市', '鄉鎮市區', '村里'], columns = ['主行業別'], values = ['綜合指標']).fillna(0)
e_inv_ratio.columns = ['_'.join(col) for col in e_inv_ratio.columns.values]
e_inv_ratio = e_inv_ratio.reset_index()


In [39]:
whitespace_remover(e_inv_ratio)

In [40]:
e_inv_ratio.columns = ['city','town','village','hotel_ind', 'retail_ind', 'ctring_ind']

In [41]:
e_inv_ratio['village'] = e_inv_ratio['village'].replace('羣賢里','群賢里')
#e_inv_ratio.query('village.str.contains("賢")', engine='python')

電子發票消費熱度指標

https://data.gov.tw/dataset/36843

https://sip.einvoice.nat.gov.tw/ods-main/ODS308E/download/691C0280-CEFB-488F-9E71-6AA4F39A41CD/1/1124193D-09F5-4711-AB9A-01848E3B88E4/0/?fileType=csv

finlab實價登錄爬蟲
https://www.finlab.tw/real-estate-analasys-histograms/

## 開始整併

In [42]:
'''
性比例	戶量	人口密度  扶養比	扶幼比	扶老比	老化指數
'''
pop_ratio.head(3)

,city,code_2,M_F_RAT,P_H_CNT,P_DEN,DEPENDENCY_RAT,A0A14_A15A65_RAT,A65UP_A15A64_RAT,A65_A0A14_RAT,INFO_TIME
0,南投縣,A0801-01,91.86,2.91,3512.41,52.33,27.77,24.56,88.45,109Y12M
1,南投縣,A0801-02,80.50,2.23,1568.15,94.48,20.04,74.44,371.56,109Y12M
2,南投縣,A0801-03,115.34,2.73,384.38,40.75,13.08,27.66,211.48,109Y12M


In [43]:
sal_dict={
  '縣市':'city',
  '鄉鎮市區':'town',
  '村里':'village',
  '平均數':'sal_mean',
  '中位數':'sal_med'
}

sal_df.columns = ['city', 'town', 'village', 'tax_unit_cnt', 'all_amt', 'sal_mean', 'sal_med', 'Q1', 'Q3', 'std', 's']
sal_df['village'] = sal_df['village'].replace('羣賢里','群賢里')

In [45]:
## 人口指標
brn_gis_df = brn_code_df.merge(pop_ratio[['code_2','P_DEN','A65UP_A15A64_RAT']], how='left', left_on='code_2', right_on='code_2')

## 年收
brn_gis_df = brn_gis_df.merge(sal_df[['city', 'town', 'village','sal_mean', 'sal_med']], how='left', on=['town','city','village'])

## 消費熱度
brn_gis_df = brn_gis_df.merge(e_inv_ratio, how='left', on=['town','city','village'])


## 公司行號工廠
brn_gis_df = brn_gis_df.merge(poi_num_df, how='left', on='brn')

## 電信信令 
brn_gis_df = brn_gis_df.merge(
  tele_df[['COUNTY','TOWN', 'day_night_ratio', 'holiday_work_ratio']],
   how='left', 
   left_on=['city','town'],
   right_on=['COUNTY','TOWN']
)



In [47]:
feat_list = [
    'P_DEN', 'A65UP_A15A64_RAT',
    'sal_mean',  
    'factory_num', 'bu_num', 'cmp_num', 'stk_num', 
    'hotel_ind', 'retail_ind', 'ctring_ind',
    'day_night_ratio','holiday_work_ratio'
]

In [48]:
arr = brn_gis_df[feat_list]
brn_gis_df[['brn','city','town','village']+feat_list].head()

,brn,city,town,village,P_DEN,A65UP_A15A64_RAT,sal_mean,factory_num,bu_num,cmp_num,stk_num,hotel_ind,retail_ind,ctring_ind,day_night_ratio,holiday_work_ratio
0,營業部(總行),臺北市,中山區,民安里,19921.56,34.61,1290,0,1369,5066,15,60.0,99.00,99.00,1.541318,0.665645
1,敦南分行,臺北市,大安區,敦安里,41990.78,34.41,2085,0,829,4887,11,73.5,89.75,80.00,1.481773,0.702457
2,新生分行,臺北市,中正區,幸市里,27008.20,36.09,2241,1,399,2574,14,0.0,92.00,91.75,1.883811,0.550355
3,新莊分行,新北市,新莊區,中華里,60972.12,15.98,1021,6,1793,1122,0,0.0,63.00,64.75,0.838921,1.183546
4,桃園分行,桃園市,桃園區,文明里,15269.66,28.40,915,19,656,637,1,0.0,55.50,60.75,0.909837,1.102585


In [49]:
brn_gis_df[brn_gis_df.isnull().any(axis=1)]

,brn,addr,lon,lat,city,town,village,code_2,code_1,code_min,eco_code3,P_DEN,A65UP_A15A64_RAT,sal_mean,sal_med,hotel_ind,retail_ind,ctring_ind,factory_num,bu_num,cmp_num,stk_num,COUNTY,TOWN,day_night_ratio,holiday_work_ratio


## Cluster

[觀光景點消費熱度分析-電子發票載具客源地區統計-資料集](https://sip.einvoice.nat.gov.tw/ods-main/ODS303E/691C0280-CEFB-488F-9E71-6AA4F39A41CD/30/Mjs=?FUNCTION_ID=ODS303E&BUILD_INFO=20211008-1333&SYSTEM_ID=ODS&SYSTEM_NAME=%E9%9A%A8%E9%81%B8&ENVIRONMENT_DISPLAY_NAME=&TITLE=%E6%AD%A1%E8%BF%8E%E8%92%9E%E8%87%A8+%E8%B2%A1%E6%94%BF%E9%83%A8%E9%9B%BB%E5%AD%90%E7%99%BC%E7%A5%A8+%E6%99%BA%E6%85%A7%E5%A5%BD%E7%94%9F%E6%B4%BB+%E6%9C%8D%E5%8B%99%E5%B9%B3%E5%8F%B0)

[電信信令人口統計之建置、分析與應用](https://ws.moi.gov.tw/Download.ashx?u=LzAwMS9VcGxvYWQvNDAwL3JlbGZpbGUvMC8xNDk0NS85NzMxZjkxNi01MzU5LTQzZDktYmVlOS0zNjMyYTUwOTcxMDYucGRm&n=6Zu75L%2Bh5L%2Bh5Luk5Lq65Y%2Bj57Wx6KiI5LmL5bu6572u44CB5YiG5p6Q6IiH5oeJ55SoLnBkZg%3D%3D&icon=..pdf)

In [55]:
fnl_df = brn_gis_df[['brn','lon','lat','city','town','village']+feat_list]

In [56]:
fnl_df.insert(0,'lon_lat', brn_gis_df['lat'].astype(str)+','+brn_gis_df['lon'].astype(str))

In [57]:
clstr_feat = [
    'P_DEN', 'A65UP_A15A64_RAT',
    'sal_mean',  
    'factory_num', 'bu_num', 'cmp_num', 'stk_num', 
    'hotel_ind', 'retail_ind', 'ctring_ind',
    'day_night_ratio','holiday_work_ratio'
]

In [58]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [59]:
clstr_arr = StandardScaler().fit_transform(fnl_df[clstr_feat])

In [60]:
pca = PCA(n_components=0.8)
pca.fit(clstr_arr)

PCA(n_components=0.8)

In [61]:
clstr_arr = pca.fit_transform(clstr_arr)

In [62]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans

In [63]:
clustering = AgglomerativeClustering(n_clusters=6).fit(clstr_arr)
#clustering = KMeans(n_clusters=5, max_iter=5000).fit(clstr_arr)


In [66]:
fnl_df.loc[:,'clstr'] = clustering.labels_

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


- 群0 住宅區、科技園區
  - 人口密度中位數低
  - 戶籍人口老化指數不高(竹科周圍人口偏年輕)
  - 周圍商家數、公司數比較少 
- 群1 北國經濟重鎮
  - 戶籍人口老化程度高
  - 周圍公司行號、上市櫃公司數爆表
- 群2 遼闊地帶
  - 路很大、很好騎
- 群3 工廠重鎮
  - 周圍工廠超級多
  - 人口密度很高
- 群4 尊榮住宅區
  - 鬧中取靜、尊榮華貴
- 群5 外圍高密度
  - 多為台北市中心外的蛋白區，戶籍人口密度高
  - 公司行號、商家數次之

In [71]:
fnl_df['clstr'].value_counts().sort_index()

0    44
1    17
2    18
3    11
4     2
5    12
Name: clstr, dtype: int64

In [72]:
brn_list = []

for i in range(0,6):
  print('群'+str(i))
  print(list(fnl_df[fnl_df['clstr']==i]['brn']))
  haha.append(fnl_df[fnl_df['clstr']==i]['brn'].reset_index(drop=True))

群0
['台中分行', '高雄分行', '台南分行', '中壢分行', '蘆洲分行', '天母分行', '大里分行', '豐原分行', '新竹分行', '北台中分行', '永福分行', '屏東分行', '龍潭分行', '崇德分行', '後甲分行', '海佃分行', '鳳山分行', '太平分行', '北高雄分行', '逢甲分行', '竹科分行', '南屯分行', '民權分行', '三民分行', '北新店分行', '府城分行', '北桃園分行', '大墩分行', '延平分行', '南崁分行', '文心分行', '岡山分行', '大雅分行', '成功分行', '石牌分行', '永康分行', '新板分行', '永華分行', '關東橋分行', '南寮分行', '市府分行', '竹南分行', '景美分行', '雄科分行']
群1
['營業部(總行)', '敦南分行', '南京東路分行', '信託部', '國外部', '國際金融業務分行', '信義分行', '建橋分行', '內湖分行', '西門分行', '敦北分行', '忠孝分行', '復興分行', '建北分行', '基隆路分行', '南松山分行', '南港分行']
群2
['桃園分行', '苓雅分行', '嘉義分行', '花蓮分行', '七賢分行', '金華分行', '彰化分行', '五甲分行', '東基隆分行', '北大分行', '竹北分行', '八德分行', '員林分行', '右昌分行', '沙鹿分行', '羅東分行', '文山分行', '副都心分行']
群3
['新生分行', '大安分行', '古亭分行', '民生分行', '和平分行', '大直分行', '南門分行', '松江分行', '北師分行', '松德分行', '東湖分行']
群4
['南新莊分行', '汐止分行']
群5
['新莊分行', '板橋分行', '中和分行', '三重分行', '永和分行', '板南分行', '三和分行', '東高雄分行', '新店分行', '景平分行', '淡水分行', '江翠分行']


In [76]:
#pd.DataFrame(pd.concat(haha,axis=0,sort=True))

In [75]:
agg_df = fnl_df.groupby('clstr')[feat_list].describe().T
agg_df.iloc[agg_df.index.get_level_values(1)=='mean']#.to_csv('brn_stat.csv')

,clstr,0,1,2,3,4,5
P_DEN,mean,20308.022500,24010.448824,18546.694444,52447.912727,43800.685000,48870.373333
A65UP_A15A64_RAT,mean,19.832727,29.961176,25.524444,28.000000,15.860000,24.272500
sal_mean,mean,1209.931818,1613.882353,912.444444,1535.545455,769.500000,977.000000
factory_num,mean,9.704545,4.117647,7.111111,0.727273,133.000000,13.166667
bu_num,mean,657.090909,834.705882,1093.277778,571.818182,836.500000,1991.750000
cmp_num,mean,683.204545,4874.588235,528.666667,2013.181818,875.000000,1050.583333
stk_num,mean,1.227273,29.117647,0.444444,6.818182,2.000000,1.000000
hotel_ind,mean,36.596591,63.970588,33.777778,15.750000,0.000000,9.145833
retail_ind,mean,87.767045,94.602941,52.111111,85.545455,84.750000,51.708333
ctring_ind,mean,89.676136,94.352941,59.000000,85.068182,87.500000,65.041667
